# Evaluation Framework Playground

Interactive notebook for evaluating agent performance with the cascade orchestrator.

## Features

- ✅ Test YAML scenario loading
- ✅ Run orchestrator turns with full evaluation
- ✅ Record events and score performance
- ✅ Compare model configurations (GPT-4o vs o1, different verbosity levels)
- ✅ A/B testing capabilities

## Quick Links

- [Evaluation Package](../../../apps/artagent/backend/evaluation/)
- [Documentation](../../../docs/testing/model-evals.md)
- [Test Scenarios](../../../tests/eval_scenarios/)

## Setup

In [13]:
# Import required libraries
import asyncio
import os
import sys
from pathlib import Path
import json
import yaml
from IPython.display import display, HTML, Markdown
import ipywidgets as widgets

# Add project root to Python path
PROJECT_ROOT = Path(os.getcwd()).parent.parent.parent
sys.path.insert(0, str(PROJECT_ROOT))

print(f"📁 Project Root: {PROJECT_ROOT}")
print(f"✅ Python path configured")

📁 Project Root: /Users/jinle/Repos/_AIProjects/art-voice-agent-accelerator
✅ Python path configured

✅ Python path configured


## Load Environment

In [14]:
# Load environment configuration
import os
from pathlib import Path
from dotenv import load_dotenv

# Load .env first (fallback)
env_local_path = PROJECT_ROOT / ".env.local"
env_path = PROJECT_ROOT / ".env"

if env_local_path.exists():
    print(f"✅ Loading .env.local")
    load_dotenv(env_local_path, override=True)
    config_source = ".env.local"
elif env_path.exists():
    print(f"✅ Loading .env")
    load_dotenv(env_path, override=True)
    config_source = ".env"
else:
    print("⚠️  No .env file found. Using system environment variables.")
    config_source = "system environment"

# Try to load Azure App Configuration (preferred)
try:
    from apps.artagent.backend.config.appconfig_provider import bootstrap_appconfig, get_provider_status
    
    appconfig_loaded = bootstrap_appconfig()
    if appconfig_loaded:
        status = get_provider_status()
        endpoint_name = status.get("endpoint", "").split("//")[-1].split(".")[0] if status.get("endpoint") else "unknown"
        print(f"✅ Loaded configuration from Azure App Config ({endpoint_name})")
        config_source = f"Azure App Config ({endpoint_name})"
except Exception as e:
    print(f"ℹ️  App Configuration not available, using {config_source}")

# Verify Azure OpenAI is configured
endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
deployment = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_ID') or 'gpt-4o'

print(f"\n📋 Configuration source: {config_source}")
if endpoint:
    print(f"✅ Azure OpenAI endpoint: {endpoint}")
else:
    print("❌ AZURE_OPENAI_ENDPOINT not set")

if deployment:
    print(f"✅ Default deployment: {deployment}")
else:
    print("⚠️  AZURE_OPENAI_DEPLOYMENT_NAME not set")

✅ Loading .env.local


   App Config (appconfig-contoso-z8kttnsm): 50 keys synced


✅ Loaded configuration from Azure App Config (appconfig-contoso-z8kttnsm)

📋 Configuration source: Azure App Config (appconfig-contoso-z8kttnsm)
✅ Azure OpenAI endpoint: https://artagentz8kttnsmaif.openai.azure.com/
✅ Default deployment: gpt-4o


📋 Configuration source: Azure App Config (appconfig-contoso-z8kttnsm)
✅ Azure OpenAI endpoint: https://artagentz8kttnsmaif.openai.azure.com/
✅ Default deployment: gpt-4o


## Import Components

In [15]:
# Import evaluation framework
from apps.artagent.backend.evaluation import (
    EventRecorder,
    EvaluationOrchestratorWrapper,
    MetricsScorer,
    ComparisonRunner,
    MockMemoManager,
    build_context,
)

# Import orchestrator components
from apps.artagent.backend.registries.agentstore.loader import (
    discover_agents,
    get_agent,
    build_handoff_map,
)
from apps.artagent.backend.registries.agentstore.base import ModelConfig
from apps.artagent.backend.voice.speech_cascade.orchestrator import (
    CascadeOrchestratorAdapter,
)
from apps.artagent.backend.voice.shared.base import OrchestratorContext

print("✅ All components imported successfully")

✅ All components imported successfully



## 1. Discover Available Agents

Load real agents from the agent registry:

In [16]:
# Discover all available agents
agents = discover_agents()
handoff_map = build_handoff_map(agents)

print(f"📦 Discovered {len(agents)} agents:\n")

# Group by category
banking = []
insurance = []
other = []

for name, agent in sorted(agents.items()):
    desc = agent.description[:80] if agent.description else "No description"
    
    # Categorize based on name patterns
    if any(x in name.lower() for x in ['banking', 'fraud', 'investment', 'card', 'auth', 'compliance']):
        banking.append((name, desc))
    elif any(x in name.lower() for x in ['claims', 'policy', 'fnol', 'subro']):
        insurance.append((name, desc))
    else:
        other.append((name, desc))

if banking:
    print("Banking Agents:")
    for name, desc in banking:
        print(f"  • {name}: {desc}")
    print()

if insurance:
    print("Insurance Agents:")
    for name, desc in insurance:
        print(f"  • {name}: {desc}")
    print()

if other:
    print("Other Agents:")
    for name, desc in other:
        print(f"  • {name}: {desc}")
    print()

print(f"\n🔗 Handoff map: {len(handoff_map)} handoff triggers configured")

📦 Discovered 12 agents:

Banking Agents:
  • AuthAgent: Handles MFA, identity verification, and security questions
  • BankingConcierge: Primary banking assistant - handles most customer needs and routes complex reque
  • CardRecommendation: Credit card recommendations, comparisons, and e-signature applications
  • ComplianceDesk: AML, FATCA, sanctions screening, and regulatory compliance verification
  • FraudAgent: Post-authentication fraud detection specialist handling credit card fraud,
ident
  • InvestmentAdvisor: Retirement accounts, 401(k) rollovers, IRA guidance, investment products

Insurance Agents:
  • ClaimsSpecialist: Insurance claims specialist who helps customers file, track, and manage their in
  • FNOLAgent: Insurance First-Notice-of-Loss (FNOL) intake specialist. Collects claim
informat
  • PolicyAdvisor: Insurance policy advisor who helps customers with policy changes, renewals, and 
  • SubroAgent: Subrogation specialist handling B2B calls from Claimant Carriers (ot

## 2. Create Custom Cascade Orchestrator instance

This creates an actual orchestrator with real agents (not mocks):

In [17]:
def create_orchestrator(
    agent_name: str,
    model_override: dict = None,
    session_id: str = "eval-session",
) -> CascadeOrchestratorAdapter:
    """
    Create a real orchestrator for evaluation.
    
    Args:
        agent_name: Name of the agent to use
        model_override: Optional model configuration override
        session_id: Session ID for tracking
    
    Returns:
        Configured CascadeOrchestratorAdapter
    """
    # Load all agents
    all_agents = discover_agents()
    
    # Apply model override if provided
    if model_override and agent_name in all_agents:
        agent = all_agents[agent_name]
        deployment_id = model_override.get('deployment_id', agent.model.deployment_id)
        
        # Detect model family from deployment_id
        deployment_lower = deployment_id.lower()
        
        # Models that use max_completion_tokens instead of max_tokens
        uses_max_completion_tokens = any(x in deployment_lower for x in [
            'o1', 'o3', 'o4',           # Reasoning models
            'gpt-5', 'gpt5',            # GPT-5 family
            'gpt-4.1', 'gpt4.1',        # GPT-4.1 family
        ])
        
        # Models that don't support custom temperature (only default=1)
        # o-series: reasoning models
        # gpt-5: only supports default temperature
        no_custom_temperature = any(x in deployment_lower for x in [
            'o1', 'o3', 'o4',           # Reasoning models
            'gpt-5', 'gpt5',            # GPT-5 family (only default temp)
        ])
        
        # For new-gen models: use max_completion_tokens, not max_tokens
        if uses_max_completion_tokens:
            max_tokens_val = None  # New models don't support max_tokens
            max_completion_tokens_val = model_override.get(
                'max_completion_tokens', 
                model_override.get('max_tokens', 4096)  # Allow fallback from max_tokens
            )
        else:
            max_tokens_val = model_override.get('max_tokens', agent.model.max_tokens)
            max_completion_tokens_val = model_override.get('max_completion_tokens')
        
        # Temperature handling
        if no_custom_temperature:
            temperature_val = None  # These models only support default (1)
        else:
            temperature_val = model_override.get('temperature', agent.model.temperature)
        
        # Reasoning effort only for o-series
        is_reasoning_model = any(x in deployment_lower for x in ['o1', 'o3', 'o4'])
        reasoning_effort = model_override.get('reasoning_effort', 'medium') if is_reasoning_model else model_override.get('reasoning_effort')
        
        # Create new model config with proper parameters for model type
        model_config = ModelConfig(
            deployment_id=deployment_id,
            endpoint_preference=model_override.get('endpoint_preference', agent.model.endpoint_preference),
            verbosity=model_override.get('verbosity', agent.model.verbosity),
            temperature=temperature_val,
            max_tokens=max_tokens_val,
            max_completion_tokens=max_completion_tokens_val,
            reasoning_effort=reasoning_effort,
        )
        
        # Update agent's model config
        agent.model = model_config
        agent.cascade_model = model_config
    
    # Build handoff map
    handoff_map = build_handoff_map(all_agents)
    
    # Create orchestrator
    orchestrator = CascadeOrchestratorAdapter.create(
        start_agent=agent_name,
        session_id=session_id,
        call_connection_id=f"eval-{session_id}",
        agents=all_agents,
        handoff_map=handoff_map,
        enable_rag=False,  # Disable RAG for faster eval
        streaming=False,   # Non-streaming for eval
    )
    
    return orchestrator

print("✅ create_orchestrator() function defined")
print("   - o-series/gpt-5: max_completion_tokens, NO temperature")
print("   - gpt-4.1: max_completion_tokens, with temperature")
print("   - gpt-4o: max_tokens + temperature (legacy)")
print("\nYou can now create orchestrators with:")
print("  orchestrator = create_orchestrator('FraudAgent')")

✅ create_orchestrator() function defined
   - o-series/gpt-5: max_completion_tokens, NO temperature
   - gpt-4.1: max_completion_tokens, with temperature
   - gpt-4o: max_tokens + temperature (legacy)

You can now create orchestrators with:
  orchestrator = create_orchestrator('FraudAgent')

   - o-series/gpt-5: max_completion_tokens, NO temperature
   - gpt-4.1: max_completion_tokens, with temperature
   - gpt-4o: max_tokens + temperature (legacy)

You can now create orchestrators with:
  orchestrator = create_orchestrator('FraudAgent')


## 3. Run a Single Turn (Real Agent!)

Test a real agent with a real query:

In [ ]:
async def run_single_turn(
    agent_name: str,
    user_query: str,
    model_override: dict = None,
    record_events: bool = True,
) -> dict:
    """
    Run a single turn with the cascade orchestrator.
    
    Args:
        agent_name: Agent to use
        user_query: User's question/request
        model_override: Optional model config override
        record_events: Whether to record events
    
    Returns:
        Result dictionary with response and metrics
    """
    import time
    from apps.artagent.backend.evaluation import MockMemoManager
    
    session_id = f"eval-{int(time.time())}"
    
    # Create MemoManager for session state
    memo_manager = MockMemoManager(
        session_id=session_id,
        context={"caller_name": "Test User", "client_id": "test_client"}
    )
    
    # Create orchestrator
    orchestrator = create_orchestrator(
        agent_name=agent_name,
        model_override=model_override,
        session_id=session_id,
    )
    
    # DEBUG: Verify model config is correctly applied on the orchestrator's agent
    if model_override:
        # Get the agent from the orchestrator's internal state (not a fresh discover)
        agent = orchestrator.agents.get(agent_name)
        if agent:
            actual_model = agent.get_model_for_mode("cascade")
            print(f"  ➡️  Model config applied: deployment={actual_model.deployment_id}, "
                  f"verbosity={actual_model.verbosity}, "
                  f"endpoint_pref={actual_model.endpoint_preference}")
    
    # Optionally wrap with recorder
    if record_events:
        recorder = EventRecorder(
            run_id=f"{agent_name}_{session_id}",
            output_dir=PROJECT_ROOT / "runs" / "jupyter_tests",
        )
        orchestrator = EvaluationOrchestratorWrapper(
            orchestrator=orchestrator,
            recorder=recorder,
        )
    
    # Create context WITH MemoManager!
    context = OrchestratorContext(
        session_id=session_id,
        user_text=user_query,
        turn_id="turn_1",
        conversation_history=memo_manager.get_history(agent_name),
        metadata={
            "scenario": "jupyter_test",
            "memo_manager": memo_manager,  # ← THE KEY!
        },
    )
    
    # Run turn through orchestrator
    start_time = time.time()
    result = await orchestrator.process_turn(context)
    elapsed_ms = (time.time() - start_time) * 1000
    
    # Update history
    memo_manager.append_to_history(agent_name, "user", user_query)
    memo_manager.append_to_history(agent_name, "assistant", result.response_text)
    
    return {
        "query": user_query,
        "agent": agent_name,
        "response": result.response_text,
        "model": model_override.get('deployment_id') if model_override else 'default',
        "endpoint": model_override.get('endpoint_preference') if model_override else 'auto',
        "verbosity": model_override.get('verbosity') if model_override else 'default',
        "input_tokens": result.input_tokens,
        "output_tokens": result.output_tokens,
        "latency_ms": elapsed_ms,
        "error": result.error,
    }

print("✅ run_single_turn() function defined")
print("   Evaluates agent with full orchestrator execution")
print("   Now includes verbosity debugging output")

✅ run_single_turn() function defined
   Evaluates agent with full orchestrator execution

   Evaluates agent with full orchestrator execution


### Test It! Run a Query

In [ ]:
# Example: Evaluate FraudAgent with GPT-4o
result = await run_single_turn(
    agent_name="FraudAgent",
    user_query="I see a $500 charge from Amazon that I didn't make",
    model_override={
        "deployment_id": "gpt-4o",
        "endpoint_preference": "responses",  # Use responses API for verbosity support
        "verbosity": 0,  # Minimal verbosity for faster responses
        "temperature": 0.7,
    },
    record_events=True,
)

# Display result
print("\n" + "=" * 70)
print("🎯 EVALUATION RESULT")
print("=" * 70)
print(f"\n📝 Query: {result['query']}")
print(f"\n🤖 Agent: {result['agent']}")
print(f"💬 Response:\n{result['response']}")
print(f"\n📊 Metrics:")
print(f"  • Model: {result['model']}")
print(f"  • Endpoint: {result['endpoint']}")
print(f"  • Verbosity: {result['verbosity']}")
print(f"  • Input tokens: {result['input_tokens']}")
print(f"  • Output tokens: {result['output_tokens']}")
print(f"  • Latency: {result['latency_ms']:.0f}ms")
if result['error']:
    print(f"  • ❌ Error: {result['error']}")
print("=" * 70)

[2026-01-02 13:56:11,907] INFO - apps.artagent.backend.evaluation.recorder: EventRecorder initialized | run_id=FraudAgent_eval-1767383771 output=/Users/jinle/Repos/_AIProjects/art-voice-agent-accelerator/runs/jupyter_tests/FraudAgent_eval-1767383771_events.jsonl
[2026-01-02 13:56:11,909] INFO - apps.artagent.backend.evaluation.wrappers: EvaluationOrchestratorWrapper initialized | orchestrator=CascadeOrchestratorAdapter recorder=FraudAgent_eval-1767383771
[2026-01-02 13:56:11,919] INFO - voice.shared.config_resolver: Checking for session-scoped scenario | session_id=eval-1767383771 scenario_name=None
[2026-01-02 13:56:11,921] INFO - voice.shared.config_resolver: No stored scenarios for session | session_id=eval-1767383771
[2026-01-02 13:56:11,970] INFO - cascade.adapter: 🔧 Agent tools loaded | agent=FraudAgent tool_count=12 tool_names=['analyze_recent_transactions', 'check_suspicious_activity', 'block_card_emergency', 'create_fraud_case', 'create_transaction_dispute', 'ship_replacement_


🎯 EVALUATION RESULT

📝 Query: I see a $500 charge from Amazon that I didn't make
🎯 EVALUATION RESULT

📝 Query: I see a $500 charge from Amazon that I didn't make

🤖 Agent: FraudAgent
💬 Response:
I found the charge and other flagged activity. Would you like me to block your card?

📊 Metrics:
  • Model: gpt-4o
  • Endpoint: chat
  • Input tokens: 0
  • Output tokens: 30
  • Latency: 2022ms


## 4. Compare Model Configurations (i.e gpt-5-mini vs gpt-4o from the above)

Test the same query with different model configurations:

In [23]:
async def compare_model_configs(
    agent_name: str,
    user_query: str,
    configs: list[dict],
) -> list[dict]:
    """
    Run the same query with different model configurations.
    
    Args:
        agent_name: Agent to test
        user_query: Query to test
        configs: List of model config dictionaries
    
    Returns:
        List of results
    """
    results = []
    
    for config in configs:
        print(f"Testing {config.get('deployment_id')} with {config.get('endpoint_preference')}...")
        
        result = await run_single_turn(
            agent_name=agent_name,
            user_query=user_query,
            model_override=config,
            record_events=True,
        )
        results.append(result)
    
    return results

# Example: Compare GPT-4o Chat vs Responses API
comparison_results = await compare_model_configs(
    agent_name="FraudAgent",
    user_query="I see a suspicious $500 charge",
    configs=[
        {
            "deployment_id": "gpt-5-nano",
            "endpoint_preference": "responses",
            "verbosity": 1,
            "temperature": 0.7,
        },
        {
            "deployment_id": "gpt-5-nano",
            "endpoint_preference": "responses",
            "verbosity": 0,  # Minimal
        },
        {
            "deployment_id": "gpt-5-nano",
            "endpoint_preference": "responses",
            "verbosity": 2,  # Detailed
        },
    ],
)

# Display comparison
print("\n" + "=" * 70)
print("📊 MODEL CONFIGURATION COMPARISON")
print("=" * 70)
for i, result in enumerate(comparison_results, 1):
    print(f"\n{i}. {result['model']} | {result['endpoint']}")
    print(f"   Response length: {len(result['response'])} chars")
    print(f"   Output tokens: {result['output_tokens']}")
    print(f"   Latency: {result['latency_ms']:.0f}ms")
    print(f"   Response: {result['response'][:150]}...")

print("\n" + "=" * 70)

Testing gpt-5-nano with responses...


[2026-01-02 14:04:04,374] INFO - apps.artagent.backend.evaluation.recorder: EventRecorder initialized | run_id=FraudAgent_eval-1767384244 output=/Users/jinle/Repos/_AIProjects/art-voice-agent-accelerator/runs/jupyter_tests/FraudAgent_eval-1767384244_events.jsonl
[2026-01-02 14:04:04,380] INFO - apps.artagent.backend.evaluation.wrappers: EvaluationOrchestratorWrapper initialized | orchestrator=CascadeOrchestratorAdapter recorder=FraudAgent_eval-1767384244
[2026-01-02 14:04:04,397] INFO - voice.shared.config_resolver: Checking for session-scoped scenario | session_id=eval-1767384244 scenario_name=None
[2026-01-02 14:04:04,400] INFO - voice.shared.config_resolver: No stored scenarios for session | session_id=eval-1767384244
[2026-01-02 14:04:04,526] INFO - cascade.adapter: 🔧 Agent tools loaded | agent=FraudAgent tool_count=12 tool_names=['analyze_recent_transactions', 'check_suspicious_activity', 'block_card_emergency', 'create_fraud_case', 'create_transaction_dispute', 'ship_replacement_

Testing gpt-5-nano with responses...


[2026-01-02 14:04:31,714] INFO - apps.artagent.backend.evaluation.recorder: EventRecorder initialized | run_id=FraudAgent_eval-1767384271 output=/Users/jinle/Repos/_AIProjects/art-voice-agent-accelerator/runs/jupyter_tests/FraudAgent_eval-1767384271_events.jsonl
[2026-01-02 14:04:31,717] INFO - apps.artagent.backend.evaluation.wrappers: EvaluationOrchestratorWrapper initialized | orchestrator=CascadeOrchestratorAdapter recorder=FraudAgent_eval-1767384271
[2026-01-02 14:04:31,728] INFO - voice.shared.config_resolver: Checking for session-scoped scenario | session_id=eval-1767384271 scenario_name=None
[2026-01-02 14:04:31,731] INFO - voice.shared.config_resolver: No stored scenarios for session | session_id=eval-1767384271
[2026-01-02 14:04:31,894] INFO - cascade.adapter: 🔧 Agent tools loaded | agent=FraudAgent tool_count=12 tool_names=['analyze_recent_transactions', 'check_suspicious_activity', 'block_card_emergency', 'create_fraud_case', 'create_transaction_dispute', 'ship_replacement_

Testing gpt-5-nano with responses...


[2026-01-02 14:05:31,806] INFO - apps.artagent.backend.evaluation.recorder: EventRecorder initialized | run_id=FraudAgent_eval-1767384331 output=/Users/jinle/Repos/_AIProjects/art-voice-agent-accelerator/runs/jupyter_tests/FraudAgent_eval-1767384331_events.jsonl
[2026-01-02 14:05:31,808] INFO - apps.artagent.backend.evaluation.wrappers: EvaluationOrchestratorWrapper initialized | orchestrator=CascadeOrchestratorAdapter recorder=FraudAgent_eval-1767384331
[2026-01-02 14:05:31,820] INFO - voice.shared.config_resolver: Checking for session-scoped scenario | session_id=eval-1767384331 scenario_name=None
[2026-01-02 14:05:31,822] INFO - voice.shared.config_resolver: No stored scenarios for session | session_id=eval-1767384331
[2026-01-02 14:05:31,942] INFO - cascade.adapter: 🔧 Agent tools loaded | agent=FraudAgent tool_count=12 tool_names=['analyze_recent_transactions', 'check_suspicious_activity', 'block_card_emergency', 'create_fraud_case', 'create_transaction_dispute', 'ship_replacement_


📊 MODEL CONFIGURATION COMPARISON

📊 MODEL CONFIGURATION COMPARISON

1. gpt-5-nano | responses
   Response length: 381 chars
   Output tokens: 95
   Latency: 27183ms
   Response: I found two suspicious charges.

The first is five thousand dollars at WIRE TRANSFER INTL. Date: 2024-11-29. Flags: wire_transfer, unusual_amount, fir...

2. gpt-5-nano | responses
   Response length: 463 chars
   Output tokens: 115
   Latency: 59940ms
   Response: Thanks for telling me. I found two suspicious transactions. One is one thousand eight hundred ninety-nine dollars and ninety-nine cents at ELECTRONICS...

3. gpt-5-nano | responses
   Response length: 0 chars
   Output tokens: 0
   Latency: 55723ms
   Response: ...



## Diagnostic: Verify Model Parameters

Let's verify what parameters are actually being sent to the API:

In [ ]:
# Diagnostic: Check what parameters are being prepared for the API call
from src.aoai.manager import AzureOpenAIManager
from apps.artagent.backend.registries.agentstore.base import ModelConfig

def diagnose_model_params(model_override: dict):
    """
    Diagnose what parameters would be sent to the API.
    
    This shows exactly what the AzureOpenAIManager prepares
    based on your model_override settings.
    """
    # Create a ModelConfig from the override
    config = ModelConfig(
        deployment_id=model_override.get('deployment_id', 'gpt-4o'),
        endpoint_preference=model_override.get('endpoint_preference', 'auto'),
        verbosity=model_override.get('verbosity', 0),
        temperature=model_override.get('temperature'),
        max_tokens=model_override.get('max_tokens'),
        max_completion_tokens=model_override.get('max_completion_tokens'),
        reasoning_effort=model_override.get('reasoning_effort'),
    )
    
    print(f"📋 ModelConfig created:")
    print(f"   • deployment_id: {config.deployment_id}")
    print(f"   • endpoint_preference: {config.endpoint_preference}")
    print(f"   • verbosity: {config.verbosity}")
    print(f"   • temperature: {config.temperature}")
    print(f"   • max_tokens: {config.max_tokens}")
    print(f"   • max_completion_tokens: {config.max_completion_tokens}")
    print(f"   • reasoning_effort: {config.reasoning_effort}")
    
    # Create a temporary manager to check parameter preparation
    manager = AzureOpenAIManager(enable_tracing=False)
    
    # Check which endpoint would be used
    use_responses = manager._should_use_responses_endpoint(config)
    print(f"\n🔀 Endpoint decision: {'RESPONSES' if use_responses else 'CHAT'}")
    
    # Sample messages for testing
    test_messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello"},
    ]
    
    # Get prepared parameters
    if use_responses:
        params = manager._prepare_responses_params(config, test_messages)
        print(f"\n📦 Responses API params:")
    else:
        params = manager._prepare_chat_params(config, test_messages)
        print(f"\n📦 Chat API params:")
    
    # Show key parameters (excluding messages for brevity)
    for key, value in params.items():
        if key != 'messages' and key != 'input':
            print(f"   • {key}: {value}")
    
    # Check if verbosity is included
    if 'verbosity' in params:
        print(f"\n✅ verbosity IS being sent to API: {params['verbosity']}")
    else:
        print(f"\n⚠️  verbosity is NOT in the API params!")
    
    return params

# Test with different configurations
print("=" * 70)
print("🔍 DIAGNOSTIC: Testing verbosity parameter passing")
print("=" * 70)

print("\n--- Test 1: GPT-4o with Chat endpoint ---")
diagnose_model_params({
    "deployment_id": "gpt-4o",
    "endpoint_preference": "chat",
    "verbosity": 0,
})

print("\n--- Test 2: GPT-4o with Responses endpoint ---")
diagnose_model_params({
    "deployment_id": "gpt-4o",
    "endpoint_preference": "responses",
    "verbosity": 0,
})

print("\n--- Test 3: GPT-5-mini with Responses endpoint ---")
diagnose_model_params({
    "deployment_id": "gpt-5-mini",
    "endpoint_preference": "responses",
    "verbosity": 0,
    "reasoning_effort": "low",
})

print("\n" + "=" * 70)

## 5. Score Recorded Events

Load and score events that were recorded:

In [9]:
# Find recorded events
runs_dir = PROJECT_ROOT / "runs" / "jupyter_tests"

if runs_dir.exists():
    event_files = list(runs_dir.glob("*_events.jsonl"))
    
    if event_files:
        print(f"📂 Found {len(event_files)} event file(s):\n")
        for f in event_files:
            print(f"  • {f.name}")
        
        # Score the most recent one
        latest_events = event_files[-1]
        print(f"\n📊 Scoring: {latest_events.name}\n")
        
        # Load and score
        scorer = MetricsScorer()
        events = scorer.load_events(latest_events)
        
        # Generate summary
        summary = scorer.generate_summary(
            events,
            scenario_name="jupyter_test",
        )
        
        # Display summary
        print("=" * 60)
        print(f"Scenario: {summary.scenario_name}")
        print(f"Agent: {summary.agent_name}")
        print(f"Total Turns: {summary.total_turns}")
        print(f"\n🔧 Tool Metrics:")
        print(f"  Precision: {summary.tool_metrics['precision']:.2%}")
        print(f"  Recall: {summary.tool_metrics['recall']:.2%}")
        print(f"  Efficiency: {summary.tool_metrics['efficiency']:.2%}")
        print(f"\n⏱️  Latency:")
        print(f"  P50: {summary.latency_metrics['e2e_p50_ms']:.1f}ms")
        print(f"  P95: {summary.latency_metrics['e2e_p95_ms']:.1f}ms")
        print(f"\n💰 Cost:")
        total_tokens = summary.cost_analysis['total_input_tokens'] + summary.cost_analysis['total_output_tokens']
        print(f"  Total tokens: {total_tokens:,}")
        print(f"  Estimated cost: ${summary.cost_analysis['estimated_cost_usd']:.4f}")
        print("=" * 60)
    else:
        print("⚠️  No event files found. Run some tests first!")
else:
    print("⚠️  Runs directory not found. Run some tests first!")

📂 Found 45 event file(s):

  • AuthAgent_eval-1767383325_events.jsonl
  • AuthAgent_eval-1767302973_events.jsonl
  • FraudAgent_eval-1767382517_events.jsonl
  • FraudAgent_eval-1767381582_events.jsonl
  • FraudAgent_eval-1767381654_events.jsonl
  • FraudAgent_eval-1767381606_events.jsonl
  • FraudAgent_eval-1767381769_events.jsonl
  • FraudAgent_eval-1767304683_events.jsonl
  • AuthAgent_eval-1767305809_events.jsonl
  • FraudAgent_eval-1767381697_events.jsonl
  • AuthAgent_eval-1767306630_events.jsonl
  • FraudAgent_eval-1767304682_events.jsonl
  • FraudAgent_eval-1767303785_events.jsonl
  • FraudAgent_eval-1767303782_events.jsonl
  • FraudAgent_eval-1767382508_events.jsonl
  • FraudAgent_eval-1767304659_events.jsonl
  • FraudAgent_eval-1767381759_events.jsonl
  • FraudAgent_eval-1767383708_events.jsonl
  • FraudAgent_eval-1767382518_events.jsonl
  • FraudAgent_eval-1767381809_events.jsonl
  • FraudAgent_eval-1767303787_events.jsonl
  • FraudAgent_eval-1767303993_events.jsonl
  • AuthA

[2026-01-02 13:55:08,536] INFO - apps.artagent.backend.evaluation.scorer: Loaded 1 events from /Users/jinle/Repos/_AIProjects/art-voice-agent-accelerator/runs/jupyter_tests/FraudAgent_eval-1767304143_events.jsonl


Scenario: jupyter_test
Agent: FraudAgent
Total Turns: 1

🔧 Tool Metrics:
  Precision: 100.00%
  Recall: 100.00%
  Efficiency: 100.00%

⏱️  Latency:
  P50: 624.7ms
  P95: 624.7ms

💰 Cost:
  Total tokens: 0
  Estimated cost: $0.0000
Scenario: jupyter_test
Agent: FraudAgent
Total Turns: 1

🔧 Tool Metrics:
  Precision: 100.00%
  Recall: 100.00%
  Efficiency: 100.00%

⏱️  Latency:
  P50: 624.7ms
  P95: 624.7ms

💰 Cost:
  Total tokens: 0
  Estimated cost: $0.0000


## 6. Interactive Widget for Testing

A simple widget to test different agents and queries:

In [10]:
# Create interactive widgets (fresh instances each run to avoid handler accumulation)
agent_selector = widgets.Dropdown(
    options=sorted(list(agents.keys())),
    description='Agent:',
    style={'description_width': '100px'},
)

model_selector = widgets.Dropdown(
    options=[
        # GPT-4 family
        'gpt-4o',
        'gpt-4o-mini',
        'gpt-4.1',
        'gpt-4.1-mini',
        'gpt-4.1-nano',
        # GPT-5 family
        'gpt-5',
        'gpt-5-chat',
        'gpt-5-mini',
        # o-series (reasoning models)
        'o1',
        'o1-mini',
        'o1-preview',
        'o3',
        'o3-mini',
        'o4-mini',
        'gpt-oss-120b'
    ],
    value='gpt-4o',
    description='Model:',
    style={'description_width': '100px'},
)

endpoint_selector = widgets.Dropdown(
    options=['auto', 'chat', 'responses'],
    value='auto',
    description='Endpoint:',
    style={'description_width': '100px'},
)

query_input = widgets.Textarea(
    value='I need help with my account',
    description='Query:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='500px', height='80px')
)

# Create fresh button each run (avoids accumulating click handlers)
test_button = widgets.Button(
    description='Run Test',
    button_style='success',
    icon='play',
)

output_area = widgets.Output()

# Apply nest_asyncio once at module level
import nest_asyncio
nest_asyncio.apply()

def on_button_click(b):
    """Handle button click - run async test."""
    with output_area:
        output_area.clear_output(wait=True)
        print("⏳ Running test...\n")
        
        try:
            # Get the current event loop and run the coroutine
            loop = asyncio.get_event_loop()
            result = loop.run_until_complete(run_single_turn(
                agent_name=agent_selector.value,
                user_query=query_input.value,
                model_override={
                    "deployment_id": model_selector.value,
                    "endpoint_preference": endpoint_selector.value,
                },
            ))
            
            print("✅ Test Complete\n")
            print(f"Agent: {result['agent']}")
            print(f"Model: {result['model']} ({result['endpoint']})\n")
            print(f"Response:\n{result['response']}\n")
            print(f"Tokens: {result['output_tokens']} | Latency: {result['latency_ms']:.0f}ms")
            
            if result['error']:
                print(f"\n⚠️ Error: {result['error']}")
        
        except Exception as e:
            print(f"❌ Error: {e}")
            import traceback
            traceback.print_exc()

# Register handler on fresh button
test_button.on_click(on_button_click)

# Build and display widget UI
widget_ui = widgets.VBox([
    widgets.HTML("<h3>🧪 Interactive Agent Tester</h3>"),
    agent_selector,
    model_selector,
    endpoint_selector,
    query_input,
    test_button,
    output_area,
])

display(widget_ui)

## 7. Load YAML Comparison

Load the fraud detection comparison YAML:

In [11]:
# Load comparison YAML
comparison_path = PROJECT_ROOT / "tests" / "eval_scenarios" / "ab_tests" / "fraud_detection_comparison.yaml"

if comparison_path.exists():
    runner = ComparisonRunner(
        comparison_path=comparison_path,
        output_dir=PROJECT_ROOT / "runs" / "jupyter_comparison",
    )
    
    print(f"✅ Loaded: {runner.comparison['comparison_name']}")
    print(f"\nVariants:")
    for variant in runner.comparison['variants']:
        print(f"  • {variant['variant_id']}")
        model = variant.get('model_override', {})
        print(f"    Model: {model.get('deployment_id')}")
        print(f"    Endpoint: {model.get('endpoint_preference')}")
    
    print(f"\nNote: To run this comparison with real orchestrators,")
    print(f"      you'd need to implement the full comparison runner.")
    print(f"      For now, use the compare_model_configs() function above.")
else:
    print("⚠️  Comparison YAML not found")

[2026-01-02 13:55:08,625] INFO - apps.artagent.backend.evaluation.scenario_runner: Loading comparison from: /Users/jinle/Repos/_AIProjects/art-voice-agent-accelerator/tests/eval_scenarios/ab_tests/fraud_detection_comparison.yaml
[2026-01-02 13:55:08,649] INFO - apps.artagent.backend.evaluation.scenario_runner: Loaded comparison: gpt4o_vs_o1_fraud


✅ Loaded: gpt4o_vs_o1_fraud

Variants:
  • gpt4o_baseline
    Model: gpt-4o
    Endpoint: chat
  • o3_mini_responses
    Model: o3-mini
    Endpoint: responses

Note: To run this comparison with real orchestrators,
      you'd need to implement the full comparison runner.
      For now, use the compare_model_configs() function above.


Variants:
  • gpt4o_baseline
    Model: gpt-4o
    Endpoint: chat
  • o3_mini_responses
    Model: o3-mini
    Endpoint: responses

Note: To run this comparison with real orchestrators,
      you'd need to implement the full comparison runner.
      For now, use the compare_model_configs() function above.


## Next Steps

You now have:

✅ **Real agent testing** - Not mocks, actual orchestrator
✅ **Event recording** - All turns recorded to JSONL
✅ **Metrics scoring** - Performance analysis
✅ **Model comparison** - Test different configs
✅ **Real-time optimization** - Best practices for minimal latency

### Try These:

1. **Test different agents**:
   ```python
   result = await run_single_turn(
       agent_name="InvestmentAdvisor",
       user_query="How's my 401k doing?",
   )
   ```

2. **Compare endpoints**:
   ```python
   results = await compare_model_configs(
       agent_name="FraudAgent",
       user_query="Suspicious charge",
       configs=[{...}, {...}],
   )
   ```

3. **Analyze recorded events**:
   - Events are saved to `runs/jupyter_tests/`
   - Use MetricsScorer to analyze them

4. **Optimize for real-time**:
   - See the Real-Time Optimization section above
   - Use `verbosity=0`, `reasoning_effort="low"`, and capped tokens
   - Test with the benchmark to measure improvements

### Resources

- [Evaluation Package](../../../apps/artagent/backend/evaluation/)
- [Full Documentation](../../../docs/testing/model-evals.md)
- [YAML Scenarios](../../../tests/eval_scenarios/)

In [12]:
# Real-time optimization comparison
# Compare optimal vs suboptimal configurations

import time

async def benchmark_realtime_configs():
    """
    Benchmark optimal vs suboptimal real-time configurations.
    Shows the impact of proper configuration on latency.
    """
    test_query = "I see a suspicious charge on my account"
    
    # ✅ OPTIMAL Real-Time Configuration
    optimal_config = {
        "deployment_id": "gpt-5-mini",
        "endpoint_preference": "responses",
        "verbosity": 0,  # Minimal - fastest
        "reasoning_effort": "low",  # Fast reasoning
        "max_completion_tokens": 2048,  # Reasonable limit
    }
    
    # ⚠️ SUBOPTIMAL Configuration (for comparison)
    suboptimal_config = {
        "deployment_id": "gpt-5-mini", 
        "endpoint_preference": "responses",
        "verbosity": 2,  # Detailed - slower
        # No reasoning_effort set - will use default "medium"
        "max_completion_tokens": 8192,  # Excessive for real-time
    }
    
    print("🚀 Benchmarking Real-Time Configurations\n")
    print("=" * 70)
    
    # Test 1: Optimal configuration
    print("\n✅ Testing OPTIMAL configuration (verbosity=0, low reasoning, capped tokens)...")
    start = time.time()
    result_optimal = await run_single_turn(
        agent_name="FraudAgent",
        user_query=test_query,
        model_override=optimal_config,
        record_events=False,  # Skip recording for faster benchmark
    )
    optimal_latency = (time.time() - start) * 1000
    
    # Test 2: Suboptimal configuration  
    print(f"\n⚠️  Testing SUBOPTIMAL configuration (verbosity=2, higher tokens)...")
    start = time.time()
    result_suboptimal = await run_single_turn(
        agent_name="FraudAgent",
        user_query=test_query,
        model_override=suboptimal_config,
        record_events=False,
    )
    suboptimal_latency = (time.time() - start) * 1000
    
    # Display comparison
    print("\n" + "=" * 70)
    print("📊 REAL-TIME OPTIMIZATION RESULTS")
    print("=" * 70)
    
    print(f"\n✅ OPTIMAL Configuration:")
    print(f"   • Verbosity: 0 (minimal)")
    print(f"   • Reasoning: low")
    print(f"   • Max tokens: 2048")
    print(f"   • Response length: {len(result_optimal['response'])} chars")
    print(f"   • Output tokens: {result_optimal['output_tokens']}")
    print(f"   • Latency: {optimal_latency:.0f}ms")
    
    print(f"\n⚠️  SUBOPTIMAL Configuration:")
    print(f"   • Verbosity: 2 (detailed)")
    print(f"   • Reasoning: default")
    print(f"   • Max tokens: 8192")
    print(f"   • Response length: {len(result_suboptimal['response'])} chars")
    print(f"   • Output tokens: {result_suboptimal['output_tokens']}")
    print(f"   • Latency: {suboptimal_latency:.0f}ms")
    
    # Calculate improvement
    improvement_pct = ((suboptimal_latency - optimal_latency) / suboptimal_latency) * 100
    latency_diff = suboptimal_latency - optimal_latency
    
    print(f"\n🎯 Performance Impact:")
    print(f"   • Latency reduction: {latency_diff:.0f}ms ({improvement_pct:.1f}% faster)")
    print(f"   • Token efficiency: {result_optimal['output_tokens']} vs {result_suboptimal['output_tokens']} tokens")
    
    if improvement_pct > 10:
        print(f"\n✅ CONCLUSION: Optimal configuration provides {improvement_pct:.1f}% better latency!")
        print(f"   For real-time voice, this translates to noticeably faster responses.")
    else:
        print(f"\n💡 CONCLUSION: Both configs perform similarly (~{improvement_pct:.1f}% difference)")
    
    print("=" * 70)
    
    return {
        "optimal": result_optimal,
        "suboptimal": result_suboptimal,
        "improvement_pct": improvement_pct,
    }

# Run the benchmark
print("⏳ Running real-time optimization benchmark...\n")
benchmark_results = await benchmark_realtime_configs()

⏳ Running real-time optimization benchmark...

🚀 Benchmarking Real-Time Configurations


✅ Testing OPTIMAL configuration (verbosity=0, low reasoning, capped tokens)...


[2026-01-02 13:55:08,766] INFO - voice.shared.config_resolver: Checking for session-scoped scenario | session_id=eval-1767383708 scenario_name=None
[2026-01-02 13:55:08,768] INFO - voice.shared.config_resolver: No stored scenarios for session | session_id=eval-1767383708
[2026-01-02 13:55:08,837] INFO - cascade.adapter: 🔧 Agent tools loaded | agent=FraudAgent tool_count=12 tool_names=['analyze_recent_transactions', 'check_suspicious_activity', 'block_card_emergency', 'create_fraud_case', 'create_transaction_dispute', 'ship_replacement_card', 'send_fraud_case_email', 'provide_fraud_education', 'search_knowledge_base', 'transfer_call_to_call_center', 'escalate_emergency', 'escalate_human']
[2026-01-02 13:55:08,877] INFO - cascade.adapter: Starting LLM request (streaming) | agent=FraudAgent model=gpt-5-mini temp=N/A iteration=0 tools=12
[2026-01-02 13:55:08,926] ERROR - cascade.adapter: OpenAI stream error: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API dep


⚠️  Testing SUBOPTIMAL configuration (verbosity=2, higher tokens)...


[2026-01-02 13:55:08,974] INFO - voice.shared.config_resolver: Checking for session-scoped scenario | session_id=eval-1767383708 scenario_name=None
[2026-01-02 13:55:08,976] INFO - voice.shared.config_resolver: No stored scenarios for session | session_id=eval-1767383708
[2026-01-02 13:55:09,016] INFO - cascade.adapter: 🔧 Agent tools loaded | agent=FraudAgent tool_count=12 tool_names=['analyze_recent_transactions', 'check_suspicious_activity', 'block_card_emergency', 'create_fraud_case', 'create_transaction_dispute', 'ship_replacement_card', 'send_fraud_case_email', 'provide_fraud_education', 'search_knowledge_base', 'transfer_call_to_call_center', 'escalate_emergency', 'escalate_human']
[2026-01-02 13:55:09,055] INFO - cascade.adapter: Starting LLM request (streaming) | agent=FraudAgent model=gpt-5-mini temp=N/A iteration=0 tools=12
[2026-01-02 13:55:09,102] ERROR - cascade.adapter: OpenAI stream error: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API dep



📊 REAL-TIME OPTIMIZATION RESULTS

✅ OPTIMAL Configuration:
   • Verbosity: 0 (minimal)
   • Reasoning: low
   • Max tokens: 2048
   • Response length: 147 chars
   • Output tokens: 0
   • Latency: 206ms

⚠️  SUBOPTIMAL Configuration:
   • Verbosity: 2 (detailed)
   • Reasoning: default
   • Max tokens: 8192
   • Response length: 147 chars
   • Output tokens: 0
   • Latency: 196ms

🎯 Performance Impact:
   • Latency reduction: -10ms (-5.3% faster)
   • Token efficiency: 0 vs 0 tokens

💡 CONCLUSION: Both configs perform similarly (~-5.3% difference)
